In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pandas as pd
import openai
import os
import json
from gpt3_sandbox.api.gpt import GPT
from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from AutoReasoner import *

dataset = pd.read_csv('./dataset/WikiTableQuestions/data/pristine-unseen-tables.tsv', sep='\t')

In [18]:
i = 291
print("utterance = ", dataset.iloc[i]["utterance"])
print("utterance = ", dataset.iloc[i]["targetValue"])
df = pd.read_csv(f'./dataset/WikiTableQuestions/{dataset.iloc[i]["context"]}', sep=',')
print("|".join(df.columns))
df

utterance =  which single/ep came next after die famous?
utterance =  Redroid
Single / EP|Tracks|Label|Year|Album


,Single / EP,Tracks,Label,Year,Album
0,Your Love Is Electric,Your Love Is Electric,Burn The Fire,2009,—
1,Fresh Attire Vol. 3,Crush Groovin',Wearhouse Music,2009,—
2,Left To Right,Left To Right,Destination?,2009,—
3,Breakdown,Breakdown,Burn The Fire,2009,—
4,Doin' It Right,Doin' It Right,Burn The Fire,2009,—
5,Los Angeles,Los Angeles\nLos Angeles feat. Whiskey Pete (C...,Burn The Fire,2010,The Agenda
6,Dutchie,Dutchie,Burn The Fire,2010,—
7,Ghetto Ass Bitches,Ghetto Ass Bitches,Burn The Fire,2010,—
8,Hot & Cold,Overdose,Burn The Fire,2010,—
9,The Flying Cat,The Flying Cat,Burn The Fire,2010,—


In [17]:
sort_by_col = 'Mayor'
df = df.sort_values(sort_by_col, key=natsort_keygen())
print("Reordered: ")
print(df)

Reordered: 
     Common name  District      Hebrew       Arabic Population\n(2009)  \
52     Qalansawe    Center     קלנסווה       قلنسوة             18,700   
65         Tamra     North        טמרה         طمرة             28,700   
24       Karmiel     North      כרמיאל      كرميئيل             44,100   
42      Nazareth     North        נצרת      الناصرة             72,200   
70  Tirat Carmel     Haifa   טירת כרמל  طيرة الكرمل             18,600   
..           ...       ...         ...          ...                ...   
32    Kiryat Ono  Tel Aviv  קריית אונו   كريات أونو             31,000   
45    Ness Ziona    Center    נס ציונה   نيس تسيونا             38,100   
68      Tiberias     North       טבריה        طبريا             41,300   
72         Yavne    Center        יבנה         يبنة             33,000   
48        Ofakim     South      אופקים      أوفاكيم             24,000   

    Area\n(km²)             Mayor  
52        8.417               NaN  
65       29.259  Abu el-Hij

In [9]:
shuffleDataFrame(df)

{'default':            Player  No.    Nationality        Position Years for Jazz  \
 5    Bernie Fryer   25  United States           Guard        1975-76   
 3    Derek Fisher    2  United States           Guard      2006-2007   
 1  Derrick Favors   15  United States         Forward   2011-present   
 4     Greg Foster   44  United States  Center/Forward        1995-99   
 0      Jim Farmer   30  United States           Guard        1988-89   
 2  Kyrylo Fesenko   44        Ukraine          Center        2007-11   
 7    Terry Furlow   25  United States   Guard/Forward        1979-80   
 6     Todd Fuller   52  United States          Center        1998-99   
 
              School/Club Team  
 5                         BYU  
 3        Arkansas-Little Rock  
 1                Georgia Tech  
 4                        UTEP  
 0                     Alabama  
 2  Cherkasy Monkeys (Ukraine)  
 7              Michigan State  
 6        North Carolina State  ,
 'rand shuffle 5':            Pl

KeyError: 'League\nPos.'

In [42]:
prompt = """
The columns of a database table are as follows:
Year|Recipient|Nationality|Profession|Speech

To answer the question based onthe database table "how many times is professional writer listed as the profession according to this chart?", which column should the database table sorted by?
The column is: ```
"""

# model = "davinci-codex-002-msft"
model = "text-davinci-003"
import dotenv
config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY_ms']

openai.Completion.create(engine=model,
                        prompt=prompt,
                        max_tokens=128,
                        temperature=0,
                        top_p=1,
                        frequency_penalty=0.7,
                        n=1,
                        stream=False,
                        stop='```')

<OpenAIObject text_completion id=cmpl-6XN8jM9yo4dONHICDxsthlfTzCM1i at 0x7fbde571c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Profession"
    }
  ],
  "created": 1673411793,
  "id": "cmpl-6XN8jM9yo4dONHICDxsthlfTzCM1i",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 69,
    "total_tokens": 71
  }
}

In [ ]:
# tested_configs 
ids = """nu-0
nu-6
nu-13
nu-14
nu-16
nu-19
nu-23
nu-27
nu-38
nu-40
nu-41
nu-42
nu-44
nu-45
nu-60
nu-78
nu-80
nu-87
nu-100
nu-106
nu-141
nu-151
nu-152
nu-162
nu-172
nu-181
nu-187
nu-192
nu-217
nu-218
nu-219
nu-220
nu-221
nu-227
nu-243
nu-246
nu-255
nu-263
nu-266
nu-272
nu-299
nu-307
nu-320
nu-332
nu-337
nu-371
nu-378
nu-390
nu-392
nu-406"""

In [ ]:
# 